To Do:
- Make List page have subpages (e.g., list of forms, list of forms using a field, see below), make these fileres visible after the intro paragraph
- Make list of forms using a given field with description with embeded question bank info

Thought: make a pdf editor the first step in the weaver. Leverage the name normalizer but allow folks to add and remove fields and rename. 

In [644]:
import pandas as pd
import re
import PyPDF2
import os
from os import walk
import os.path
from os import path
import shutil

import time
from datetime import date

from joblib import dump, load

In [659]:
jurs = [["MI","Michigan"],["UT","Utah"]]
default_jur = "MI"

today = date.today().strftime("%Y-%m-%d")

In [660]:
files_df = pd.read_csv("../data/processed/form_data.csv")

print(len(files_df))
display(files_df.head())

files_sim_df = pd.read_csv("../data/processed/form_sim_data.csv")

print(len(files_sim_df))
files_sim_df.head()

1528


,id,jurisdiction,source,title,group,url,filename,downloaded,pages,fields,f_per_p,reading
0,04b3a0734774c02edf8eb9056d23954aa38e96c77c3392...,UT,www.utcourts.gov,Community Service Worksheet Third District Juv...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,COMMUNITY%20SERVICE%20WORKSHEET-FRONT%20AND%20...,2021-11-11,2,['NAME 1 (predicted <i>_nonstandard_</i> w/ 1....,7.500000,10.0
1,6e420f1b3575cfd8ef94b71977da9e38252e3395a78439...,UT,www.utcourts.gov,Third District Juvenile Court Work Program Ref...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Work_Crew_Application-2007.pdf,2021-11-11,2,['*docket_number| was <i>Court Case</i> (1.00 ...,19.000000,11.0
2,2532cd2b6d3aaff8c47726a0abd168fb4e5cdb4977c065...,UT,www.utcourts.gov,Utah State District Juvenile Court Probation O...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Probation%20Order%20revised.050502.pdf,2021-11-11,5,['of the probation department or by order of t...,4.800000,15.0
3,f6a6814890f21c11524d5785d772272916ff95909b7dba...,UT,www.utcourts.gov,Adoptee's Consent to Adoption and Waiver of Ri...,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,02_Consent_Adoptee.pdf,2021-11-11,3,"['*users1_name| was <i>Name</i> (1.00 conf)', ...",14.666667,10.0
4,be656a9a361db7c3532026b0bae372e704beb160fce37e...,UT,www.utcourts.gov,Adoption Agreement,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,08_Agreement.pdf,2021-11-11,1,"['*users1_name| was <i>Name</i> (1.00 conf)', ...",19.000000,11.0


1155126


,id_1,id_2,title_2,similarity
0,e12318ce9037e92ee8e48963f550e1a3229a17a2437b9e...,eb4e35c3b9f355d63ad14ae71866d44b14e61196763553...,Answer to Complaint to Recover Possession of P...,0.979167
1,e12318ce9037e92ee8e48963f550e1a3229a17a2437b9e...,74723a905136a3e30a9eec9aa9191f5d3e4b800ae319df...,"Answer, Damage/Health Hazard to Property, Land...",0.937500
2,e12318ce9037e92ee8e48963f550e1a3229a17a2437b9e...,ee10d4fd30c1faf309bd7b1a91da0794ec8862ee91c355...,"Answer, Damage/Health Hazard to Property, Land...",0.918367
3,e12318ce9037e92ee8e48963f550e1a3229a17a2437b9e...,5345e31f71a6ac7dc730e96039d35c9fc5d2cda658332e...,"Answer, Nonpayment of Rent, Landlord-Tenant",0.918367
4,e12318ce9037e92ee8e48963f550e1a3229a17a2437b9e...,98d5e7eedefb48b57a67dcbf928893c8032c1dec85e867...,"Answer, Nonpayment of Rent, Landlord-Tenant",0.937500


In [661]:
def clean_n(n):
    if pd.notnull(n):
        return int(n)
    else:
        return 0

In [662]:
def header(title,description,path="./",wide=0):
    
    if wide==1:
        pclass = "content_wide"
    else:
        pclass = "content"
    
    html = """<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<HTML xmlns="http://www.w3.org/1999/xhtml"
      xmlns:og="http://ogp.me/ns#"
      xmlns:fb="http://www.facebook.com/2008/fbml">
<HEAD>
    <title>%s</title>
    <!-- Global site tag (gtag.js) - Google Analytics -->
    <script async src="https://www.googletagmanager.com/gtag/js?id=UA-108858221-1"></script>
    <script>
      window.dataLayer = window.dataLayer || [];
      function gtag(){dataLayer.push(arguments);}
      gtag('js', new Date());

      gtag('config', 'UA-108858221-1');
    </script>
    <meta http-equiv="Content-type" content="text/html;charset=UTF-8"/>
    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" />
    <meta name="apple-mobile-web-app-capable" content="no" />
    <link rel="apple-touch-icon" href="%simages/icon_300.png"/>
    <meta property="og:type" content="website"/>
    <meta property="og:title" content="%s"/>
    <meta property="og:description" content="%s"/>
    <meta property="og:image" content="%simages/thefuture.png"/>

    <meta name="twitter:card" content="summary_large_image">
    <meta name="twitter:site" content="@SuffolkLITLab">
    <meta name="twitter:creator" content="@SuffolkLITLab">
    <meta name="twitter:title" content="%s">
    <meta name="twitter:description" content="%s">
    <meta name="twitter:image" content="http://www.davidcolarusso.com/images/thefuture.png"/>

    <meta http-equiv="X-UA-Compatible" content="IE=edge" />
    <link rel="shortcut icon" type="image/x-icon" href="/favicon.ico">
    <link rel="apple-touch-icon" href="%simages/icon.png"/>
    <link rel="stylesheet" href="https://code.jquery.com/ui/1.11.1/themes/smoothness/jquery-ui.css">
    <link rel="stylesheet" type="text/css" href="%scss/style.css?v=%s">
    <script src="https://code.jquery.com/jquery-1.11.1.min.js"></script>
    <script src="https://code.jquery.com/jquery-1.10.2.js"></script>
    <script src="https://code.jquery.com/ui/1.11.1/jquery-ui.js"></script>

</HEAD>
<BODY BGCOLOR="#ffffff" BACKGROUND="" MARGINWIDTH="0" MARGINHEIGHT="0">
<div style="box-sizing: border-box;width:100%%;background:#d6392b;color:white;padding:15px;text-align:center;"><b>This page is not active.</b> This page is just for testing purposes. <b>DO NOT RELY ON THE CONTENT FOUND HERE!</b></div>
<div class="%s">

    <div id="icon" style="background-size: 100px 100px;background-image: url('%simages/seal.jpg');"><a href="." alt="home"><img src="%simages/space.gif" width="100px" height="100px;" border="0" alt="LIT Logo"/></a></div>
    <h1 style="text-align:center;">
        The Legal Innovation & Technology Lab's Form Explorer <sup><font size=+1>Beta</font></sup>
        <center style="margin-top:5px;">
          <span class="subtitle">@ Suffolk Law School&nbsp;<font style="font-weight:normal">Last updated %s</font></span>
        </center>
    </h1>
"""%(title,path,title,description,path,title,description,path,path,today,pclass,path,path,today)
    return html
    
def footer(path="./"):
    html = """

<div class="footer">
    <a href="/" alt="home"><img src="%simages/blue_logo.png" width="50px" align="left" border="0" alt="LIT Logo"/></a>
<font size=-1><a href="mailto:litlab@suffolk.edu">Email</a> | <a href="https://github.com/SuffolkLITLab" target="_blank">GitHub</a> | <a href="/terms">Terms &amp; Privacy</a> | <a href="/credits">Credits</a></font>
</div>

</BODY>
</HTML>
"""%(path)
    
    return html

# Home page

In [663]:
def home():
    html = """
    <div class="menu_bar">
        <p style="text-align:center;">
      <a href="https://suffolklitlab.org/" class="menu">&nbsp;LIT Lab Home&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      <a href="." class="menu" style="color:black;">&nbsp;About&nbsp;The&nbsp;Explorer&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      &nbsp;&nbsp;<span style="font-family: sans-serif;font-size: 12px;text-transform: uppercase;">Explore:</span>&nbsp;&nbsp;
      <a href="list/" class="menu">&nbsp;Lists&nbsp;</a>
      <a href="sim/" class="menu">&nbsp;Similar Forms&nbsp;</a>
      <a href="flow/" class="menu">&nbsp;Flows&nbsp;</a>
      
        </p>
    </div>

  <div class="content"><h2><a name="what" href="#what" class="anchor"></a>What is the Form Explorer?</h2>

    <div style="float:left;width:100%">
      <div class="r_img_embed" style="max-width:500px;">
        <img src="images//bigdata.png" alt="Image of the character Data as a giant standing next to the Supreme Court" width="100%"/>
        <div class="caption">Big Data &amp; the Law, h/t <a href="https://flic.kr/p/7YZubW" target="_blank">Tim Sackton</a> &amp; <a href="https://medium.com/@wtrsld/big-data-made-me-do-it-5bfc3f46871c" target="_blank">Josh Lee</a></div>
      </div>
      <p>
        Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam non erat quis augue mollis convallis. Aliquam eu varius massa. Nam convallis lacinia magna, eget aliquet orci congue at. Interdum et malesuada fames ac ante ipsum primis in faucibus. Curabitur eu nisl eu libero porttitor convallis quis ut est. Duis porttitor turpis erat, ut viverra metus fringilla sit amet. Vestibulum tincidunt nibh id libero euismod, sed congue diam lobortis. Vestibulum maximus posuere odio, eget placerat nunc efficitur quis. Proin ut odio sem. Phasellus interdum massa sed sodales eleifend. Cras condimentum, risus et volutpat tincidunt, elit justo vulputate lacus, eget hendrerit purus turpis in nisl. Fusce placerat ex ac hendrerit condimentum. Pellentesque eget auctor leo.
      </p><p>
        Vivamus porttitor posuere sodales. Maecenas id pharetra diam. Nam quis erat quis enim placerat congue et non dui. Donec tempor vehicula ante eget tempor. Pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis egestas. Sed vitae orci neque. Maecenas ultricies dignissim neque quis consectetur. In sit amet luctus erat. Pellentesque ultrices sed dui nec feugiat. In nunc ante, bibendum non molestie imperdiet, molestie vel massa. Vestibulum a dictum orci, fringilla rutrum quam. Cras pellentesque congue porttitor. Ut neque felis, tincidunt euismod porttitor sed, feugiat et ligula. Duis vehicula eleifend elit sit amet tempus. Vestibulum condimentum libero vitae justo accumsan, vel sodales erat mattis.
      </p><p>
        Nam nec turpis elementum, tempus massa eget, dictum dui. Donec rutrum nisi pretium faucibus vestibulum. Mauris in ultrices arcu. Nulla vel sem a velit pretium consequat sed vitae felis. Nunc velit tortor, varius at accumsan id, maximus at velit. In cursus metus a purus condimentum convallis. Donec ullamcorper leo eget massa iaculis vehicula. Fusce bibendum aliquam arcu semper laoreet. Quisque consectetur finibus purus sed faucibus. Sed purus ligula, malesuada et diam quis, tincidunt suscipit enim. Curabitur eget quam a tellus suscipit sollicitudin nec ut arcu. Donec bibendum urna a dapibus porttitor. Aliquam in iaculis orci. Etiam condimentum auctor elit eget aliquam. Nulla facilisi. Proin at vestibulum nunc.
      </p>
    </div>
"""    
    return html

In [664]:
html = header("Form Explorer","description goes here")+home()+footer()
text_file = open("docs/index.html", "w")
text_file.write(html)
text_file.close()

# redirects

In [665]:
def redirects(jur):
    html = """<html>
<head>
<META http-equiv="CACHE-CONTROL" CONTENT="NO-CACHE">
<meta http-equiv="refresh" content="0; url=%s" />
</head>
</html>
"""%jur
    return html

In [666]:
html = redirects(default_jur)

text_file = open("docs/sim/index.html", "w")
text_file.write(html)
text_file.close()

text_file = open("docs/flow/index.html", "w")
text_file.write(html)
text_file.close()

text_file = open("docs/list/index.html", "w")
text_file.write(html)
text_file.close()

# Similarity Viewer

In [667]:
def simview(jur,path="./"):
    html = """
    <div class="menu_bar">
        <p style="text-align:center;">
      <a href="https://suffolklitlab.org/" class="menu">&nbsp;LIT Lab Home&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      <a href="../../" class="menu">&nbsp;About&nbsp;The&nbsp;Explorer&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      &nbsp;&nbsp;<span style="font-family: sans-serif;font-size: 12px;text-transform: uppercase;">Explore:</span>&nbsp;&nbsp; <select style="width:100px;" onChange="window.location.href='../'+this.value">"""
    
    for item in jurs: 
        if item[0]==jur[0]:
            selected = "SELECTED"
        else:
            selected = ""
        html += "<option value=\"%s\" %s>%s</option>"%(item[0],selected,item[1])
        
    html += """
      </select>&nbsp;&nbsp;
      <a href="../../list/%s" class="menu">&nbsp;Lists&nbsp;</a>
      <a href="../../sim/%s" class="menu" style="color:black;">&nbsp;Similar Forms&nbsp;</a>
      <a href="../../flow/%s" class="menu">&nbsp;Flows&nbsp;</a>
      
        </p>
    </div>

    <div class="content_wide"><h2><a name="what" href="#what" class="anchor"></a>Similar Forms View (%s Court Forms)</h2>
        <p>
          Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam non erat quis augue mollis convallis. Aliquam eu varius massa. Nam convallis lacinia magna, eget aliquet orci congue at. Interdum et malesuada fames ac ante ipsum primis in faucibus. Curabitur eu nisl eu libero porttitor convallis quis ut est. Duis porttitor turpis erat, ut viverra metus fringilla sit amet. Vestibulum tincidunt nibh id libero euismod, sed congue diam lobortis. Vestibulum maximus posuere odio, eget placerat nunc efficitur quis. Proin ut odio sem. Phasellus interdum massa sed sodales eleifend. Cras condimentum, risus et volutpat tincidunt, elit justo vulputate lacus, eget hendrerit purus turpis in nisl. Fusce placerat ex ac hendrerit condimentum. Pellentesque eget auctor leo.
        </p>
        <!--
            Possible library for viewer:
            https://www.sigmajs.org/
        <iframe src="https://www.sigmajs.org/demo/index.html" width="100%%" height="700px;"></iframe>
        -->
        <iframe src="network.html" width="100%%" height="600px;" frameBorder="0"></iframe>

    </div>
"""%(jur[0],jur[0],jur[0],jur[1])
    return html

In [668]:
for jur in jurs:    
    print(jur)
    path = "../../"
    
    if not os.path.exists("docs/sim/%s"%(jur[0])):
        os.makedirs("docs/sim/%s"%(jur[0]))
    #else:
    #    shutil.rmtree("docs/sim/%s"%(jur[0]))
    #    os.makedirs("docs/sim/%s"%(jur[0]))
        
    html = header("Form Explorer","description goes here",path,wide=1)+simview(jur,path)+footer(path)
    text_file = open("docs/sim/%s/index.html"%jur[0], "w")
    text_file.write(html)
    text_file.close()

['MI', 'Michigan']
['UT', 'Utah']


# Flow View

In [669]:
def flowview(jur,path="./"):
    html = """
    <div class="menu_bar">
        <p style="text-align:center;">
      <a href="https://suffolklitlab.org/" class="menu">&nbsp;LIT Lab Home&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      <a href="../../" class="menu">&nbsp;About&nbsp;The&nbsp;Explorer&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      &nbsp;&nbsp;<span style="font-family: sans-serif;font-size: 12px;text-transform: uppercase;">Explore:</span>&nbsp;&nbsp; <select style="width:100px;" onChange="window.location.href='../'+this.value">"""
    
    for item in jurs: 
        if item[0]==jur[0]:
            selected = "SELECTED"
        else:
            selected = ""
        html += "<option value=\"%s\" %s>%s</option>"%(item[0],selected,item[1])
        
    html += """
      </select>&nbsp;&nbsp;
      <a href="../../list/%s" class="menu">&nbsp;Lists&nbsp;</a>
      <a href="../../sim/%s" class="menu">&nbsp;Similar Forms&nbsp;</a>
      <a href="../../flow/%s" class="menu" style="color:black;">&nbsp;Flows&nbsp;</a>
        </p>
    </div>


      <div class="content_wide"><h2><a name="what" href="#what" class="anchor"></a>User Flow (%s Court Forms)</h2>
        <p>
          Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam non erat quis augue mollis convallis. Aliquam eu varius massa. Nam convallis lacinia magna, eget aliquet orci congue at. Interdum et malesuada fames ac ante ipsum primis in faucibus. Curabitur eu nisl eu libero porttitor convallis quis ut est. Duis porttitor turpis erat, ut viverra metus fringilla sit amet. Vestibulum tincidunt nibh id libero euismod, sed congue diam lobortis. Vestibulum maximus posuere odio, eget placerat nunc efficitur quis. Proin ut odio sem. Phasellus interdum massa sed sodales eleifend. Cras condimentum, risus et volutpat tincidunt, elit justo vulputate lacus, eget hendrerit purus turpis in nisl. Fusce placerat ex ac hendrerit condimentum. Pellentesque eget auctor leo.
        </p>
        <div style="overflow-x: scroll;:hidden;overflow-y: hidden;">
          <img src="../../images/.jpg" alt="" width="1500px"/>
        </div>

      </div>
"""%(jur[0],jur[0],jur[0],jur[1])
    return html

In [670]:
for jur in jurs:    
    print(jur)
    path = "../../"
    
    if not os.path.exists("docs/flow/%s"%(jur[0])):
        os.makedirs("docs/flow/%s"%(jur[0]))
    else:
        shutil.rmtree("docs/flow/%s"%(jur[0]))
        os.makedirs("docs/flow/%s"%(jur[0]))
        
    html = header("Form Explorer","description goes here",path,wide=1)+flowview(jur,path)+footer(path)
    text_file = open("docs/flow/%s/index.html"%jur[0], "w")
    text_file.write(html)
    text_file.close()

['MI', 'Michigan']
['UT', 'Utah']


# List VIew

In [671]:
pages = files_df[(files_df["jurisdiction"]==jur[0])]["pages"].mean()

In [672]:
f_per_p = files_df[(files_df["jurisdiction"]==jur[0])]["f_per_p"].mean()

In [673]:
fields = files_df[(files_df["jurisdiction"]==jur[0])]["f_per_p"].mean()*files_df[(files_df["jurisdiction"]==jur[0])]["pages"].mean()

In [674]:
reading = files_df[(files_df["jurisdiction"]==jur[0])]["reading"].mean()

In [675]:
forms= len(files_df[(files_df["jurisdiction"]==jur[0])])

In [676]:
def listview(jur,path="./"):
    html = """
    <div class="menu_bar">
        <p style="text-align:center;">
      <a href="https://suffolklitlab.org/" class="menu">&nbsp;LIT Lab Home&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      <a href="../../" class="menu">&nbsp;About&nbsp;The&nbsp;Explorer&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      &nbsp;&nbsp;<span style="font-family: sans-serif;font-size: 12px;text-transform: uppercase;">Explore:</span>&nbsp;&nbsp; <select style="width:100px;" onChange="window.location.href='../'+this.value">"""
    
    for item in jurs: 
        if item[0]==jur[0]:
            selected = "SELECTED"
        else:
            selected = ""
        html += "<option value=\"%s\" %s>%s</option>"%(item[0],selected,item[1])
        
    html += """
      </select>&nbsp;&nbsp;
      <a href="../../list/%s" class="menu" style="color:black;">&nbsp;Lists&nbsp;</a>
      <a href="../../sim/%s" class="menu">&nbsp;Similar Forms&nbsp;</a>
      <a href="../../flow/%s" class="menu">&nbsp;Flows&nbsp;</a>
        </p>
    </div>
"""%(jur[0],jur[0],jur[0])
    
    html += """
    <div class="content"><h2><a name="what" href="#what" class="anchor"></a>List of %s Court Forms</h2>

    <p>
      Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam non erat quis augue mollis convallis. Aliquam eu varius massa. Nam convallis lacinia magna, eget aliquet orci congue at. Interdum et malesuada fames ac ante ipsum primis in faucibus. Curabitur eu nisl eu libero porttitor convallis quis ut est.
    </p>
    <p>
    Below you will find %s forms. The average form is %.0f pages long, with %.0f fields per page. The average form is written at a %.2f-grade reading level.
    </p>    
    """%(jur[1],forms,pages,fields,reading)

    for cat in files_df[files_df["jurisdiction"]==jur[0]]["group"].unique():
        html += """
        <h3><a name="%s" href="#%s" class="anchor"></a>%s</h3>
        <ul>"""%(cat,cat,cat)

        for index,row in files_df[(files_df["jurisdiction"]==jur[0]) & (files_df["group"]==cat)].iterrows():
            #print(row[1][0])
            html += """<li><a href="../../form/%s/%s.html">%s</a></li>"""%(jur[0],row["id"],row["title"])
            
        html += "</ul>"

    return html

In [677]:
for jur in jurs:    
    print(jur)
    path = "../../"
    
    if not os.path.exists("docs/list/%s"%(jur[0])):
        os.makedirs("docs/list/%s"%(jur[0]))
    else:
        shutil.rmtree("docs/list/%s"%(jur[0]))
        os.makedirs("docs/list/%s"%(jur[0]))
        
    html = header("Form Explorer","description goes here",path,wide=0)+listview(jur,path)+footer(path)
    text_file = open("docs/list/%s/index.html"%(jur[0]), "w", encoding="utf-8")
    text_file.write(html)
    text_file.close()

['MI', 'Michigan']
['UT', 'Utah']


# Field Labels

In [753]:
included_fields = load('../data/processed/ML/norm_fields/included_fields.joblib') 

def fields(field_name,jur,path="./"):
    html = """
    <div class="menu_bar">
        <p style="text-align:center;">
      <a href="https://suffolklitlab.org/" class="menu">&nbsp;LIT Lab Home&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      <a href="../../" class="menu">&nbsp;About&nbsp;The&nbsp;Explorer&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      &nbsp;&nbsp;<span style="font-family: sans-serif;font-size: 12px;text-transform: uppercase;">Explore:</span>&nbsp;&nbsp; <select style="width:100px;" onChange="window.location.href='../'+this.value">"""
    
    for item in jurs: 
        if item[0]==jur[0]:
            selected = "SELECTED"
        else:
            selected = ""
        html += "<option value=\"%s\" %s>%s</option>"%(item[0],selected,item[1])
        
    html += """
      </select>&nbsp;&nbsp;
      <a href="../../list/%s" class="menu" style="color:black;">&nbsp;Lists&nbsp;</a>
      <a href="../../sim/%s" class="menu">&nbsp;Similar Forms&nbsp;</a>
      <a href="../../flow/%s" class="menu">&nbsp;Flows&nbsp;</a>
        </p>
    </div>
"""%(jur[0],jur[0],jur[0])
    
    regex = field_name+"(\|\s|$)"
    forms_with_var = files_df[(files_df["jurisdiction"]==jur[0]) & files_df['fields'].str.contains(regex)]

    html += """<div class="content"><h2><a name="title" href="#title" class="anchor"></a>%s court forms that use: <i>%s</i></h2>
      <p>
        Lorem ipsum dolor sit amet, consectetur adipiscing elit. To learn more, vist: <a href="https://suffolklitlab.org/docassemble-AssemblyLine-documentation/docs/label_variables">Field labels to use in template files</a>
      </p>
      <p>Out of %s forms, <span style="background:yellow"><b>the following %s forms use <i>%s</i>:</b></span></p>
"""%(jur[1],field_name,len(files_df[files_df["jurisdiction"]==jur[0]]),len(forms_with_var),field_name)
    
    for cat in forms_with_var["group"].unique():
        html += """
        <h3><a name="%s" href="#%s" class="anchor"></a>%s</h3>
        <ul>"""%(cat,cat,cat)

        for index,row in forms_with_var[forms_with_var["group"]==cat].iterrows():
            #print(row[1][0])
            html += """<li><a href="../../form/%s/%s.html">%s</a></li>"""%(jur[0],row["id"],row["title"])
            
        html += "</ul>"
        
    html += "</ul></div>"
    
    return html

In [754]:
for jur in jurs:  
    print(jur)
    path = "../../"
    for field in included_fields:
        print("\t"+field)
        html = header("Form Explorer","description goes here",path)+fields(field,jur,path)+footer(path)
        text_file = open("docs/list/%s/%s.html"%(jur[0],field), "w")
        text_file.write(html)
        text_file.close()

['MI', 'Michigan']
	users1_name
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	trial_court_county
	docket_number
	users1_birthdate
	signature_date
['UT', 'Utah']
	users1_name
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	trial_court_county
	docket_number
	users1_birthdate
	signature_date


# Create form pages

In [680]:
def formview(jur,path="./"):
    html = """
    <div class="menu_bar">
        <p style="text-align:center;">
      <a href="https://suffolklitlab.org/" class="menu">&nbsp;LIT Lab Home&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      <a href="../../" class="menu">&nbsp;About&nbsp;The&nbsp;Explorer&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      &nbsp;&nbsp;<span style="font-family: sans-serif;font-size: 12px;text-transform: uppercase;">Explore: """
    
    for item in jurs: 
        if item[0]==jur[0]:
            html += "%s"%item[1]

    html += """
      </span>&nbsp;&nbsp;
      <a href="../../list/%s" class="menu">&nbsp;Lists&nbsp;</a>
      <a href="../../sim/%s" class="menu">&nbsp;Similar Forms&nbsp;</a>
      <a href="../../flow/%s" class="menu">&nbsp;Flows&nbsp;</a>
        </p>
    </div>

"""%(jur[0],jur[0],jur[0])
    
    html += """
  <div class="content_wide" style="padding-left:0px;">
    <div class="pdf">
      <iframe src="https://courtformsonline.org/forms/%s.pdf" width="100%%" height="1100px;"></iframe>
    </div>
    <div class="pdfinfo">
      <h1><a name="name" href="#name" class="anchor"></a>%s</h1>
      <p>
        Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam non erat quis augue mollis convallis. Aliquam eu varius massa. Nam convallis lacinia magna, eget aliquet orci congue at. Interdum et malesuada fames ac ante ipsum primis in faucibus. Curabitur eu nisl eu libero porttitor convallis quis ut est.
      </p>
      <p>
        <b>Downloads:</b> In order to get the data on this page we scan forms for... You can download both the <a href="%s" target="_blank" class="exlink">original form</a> (last checked on %s) and the <a href="https://courtformsonline.org/forms/%s.pdf" target="_blank">machine-processed form</a> with normalized data fields.
      </p>
      <h3><a name="aboutform" href="#aboutform" class="anchor"></a>About This Form:</h3>
      <ul>"""%(row["id"],row["title"],row["url"],row["downloaded"],row["id"])
    
    html += """
        <li>Sourced from <a href="http://%s" target="_blank" class="exlink">%s</a> on %s</li>
        <li>Grouped in: <a href="../../list/%s/#%s">%s</a></li>
        <li>Page(s): %s </li>
        <li>Fields(s): %s </li>
        <li>Average fields per page: %s</li>
        <li>Reading Level: Grade %s</li>
      </ul>
      <!--
      <h3><a name="aboutusers" href="#aboutusers" class="anchor"></a>About Users of This Form:*</h3>
      <p>Filers of this form file an average of <b>X</b> other forms. <b>Y%%</b> of them identify as male.</p>
      -->
      
      <h3><a name="fields" href="#fields" class="anchor"></a>Identified Data Fields (Best Guess)<!--and Utilization:<sup>&dagger;</sup>-->:</h3>
      <ul>"""%(row["source"],row["source"],row["downloaded"],jur[0],row["group"],row["group"],clean_n(row["pages"]),clean_n(row["pages"]*row["f_per_p"]),clean_n(row["f_per_p"]),clean_n(row["reading"]))
    
    try:
        for f in eval(row["fields"]):
            html += "<li"
            if re.search('^\*',f):
                f = re.sub("^\*","",f)
                v = re.search('^(\w*)',f).groups()[0]
                f = re.sub("\|","</a>",f)
                html +=" style=\"background:#75fa7e\"><a href=\"../../list/%s/%s.html\">%s <!--100%% completed. <a href=\"\">Learn more about distribution of answers.</a>--></li>"%(jur[0],v,f)   
            else: 
                html +=">%s <!--100%% completed. <a href=\"\">Learn more about distribution of answers.</a>--></li>"%f
                
    except:
        print("")

    if not pd.isnull(row["fields"]):
        if len(eval(row["fields"]))==0:
            html+="<li>No fields found</li>"
    else:
            html+="<li>No fields found</li>"
        
    html += """
        <!--<li><a href="../form/">show more</a></li>-->
        </ul>

      </ul>"""
    
    sim_forms = files_sim_df[files_sim_df["id_1"]==row["id"]].sort_values(by=['similarity'], ascending=False)[:5]
    
    html+="""      <h3><a name="simforms" href="#simforms" class="anchor"></a>Similar forms:</h3>
      <p>For more context, visit the <a href="../../sim/%s">Similar Forms View</a>.</p>
      <ul>"""%jur[0]

    for index_2,row_2 in sim_forms.iterrows():
        html+="<li><a href=\"%s.html\">%s</a> (shares %s%% of data fields)</li>"%(row_2["id_2"],row_2["title_2"],clean_n(row_2["similarity"]*100))

    if len(sim_forms)==0:
        html+="<li>No matches found</li>"
        
    html+="""</ul>
      <!--
      <li><a href="../form/">show more</a></li>"""
        
    html += """
      <!--
      <h3><a name="cat1" href="#cat1" class="anchor"></a>Before using this form, users used:*</h3>
      <p>For more context, visit the <a href="../flow/">Flows View</a>.</p>
      <ul>
        <li><a href="../form/">Form one</a> (shares 90%% of data fields)</li>
        <li><a href="../form/">Form one</a> (shares 70%% of data fields)</li>
        <li><a href="../form/">show more</a></li>
      </ul>
      <h3><a name="cat1" href="#cat1" class="anchor"></a>After using this form, users used:*</h3>
      <p>For more context, visit the <a href="../flow/">Flows View</a>.</p>
      <ul>
        <li><a href="../form/">Form one</a> (shares 90%% of data fields)</li>
        <li><a href="../form/">Form one</a> (shares 70%% of data fields)</li>
        <li><a href="../form/">show more</a></li>
      </ul>
      -->
      <h3><a name="linter" href="#linter" class="anchor"></a>See Suggestions for Improving this Form:</h3>
      <p>Using the Document Assembly Line Linter you can...</p>
      <p>
        <a href="javascript:alert('Coming Soon!');">Run this form through the Linter</a>
      </p>
      
      <h3><a name="weaver" href="#weaver" class="anchor"></a>Create an Interactive Online Version of this Form:</h3>
      <p>Using the Document Assembly Line Weaver you can...</p>
      <p>
        <a href="https://suffolklitlab.org/docassemble-AssemblyLine-documentation/docs/generating_code" target="_blank">Run this form through the Weaver</a> (How To)
      </p>

    </div>
  </div>
    """
    return html

In [681]:
for jur in jurs:    
    if not os.path.exists("docs/form/%s"%(jur[0])):
        os.makedirs("docs/form/%s"%(jur[0]))
    else:
        shutil.rmtree("docs/form/%s"%(jur[0]))
        os.makedirs("docs/form/%s"%(jur[0]))
        

for index,row in files_df.iterrows():
    print(row["jurisdiction"],row["title"])
    
    html = header("Form Explorer","description goes here",path,wide=1)+formview([row["jurisdiction"],""],path)+footer(path)
    text_file = open("docs/form/%s/%s.html"%(row["jurisdiction"],row["id"]), "w", encoding="utf-8")
    
    text_file.write(html)
    text_file.close()

UT Community Service Worksheet Third District Juvenile Court  
UT Third District Juvenile Court Work Program Referral 
UT Utah State District Juvenile Court Probation Order 
UT Adoptee's Consent to Adoption and Waiver of Rights 
UT Adoption Agreement 
UT Adoption Decree 
UT Birth Mother's Consent to Adoption and Waiver of Rights 
UT Certificate of Readiness for Adoption Hearing 
UT Ex Parte Motion for Alternative Service 
UT Findings of Fact and Conclusions of Law 
UT Findings of Fact, Conclusions of Law, and Order on Motion for Alternative Service 
UT Guardian's Consent to Adoption and Waiver of Notice 
UT Non-custodial Father's Consent to Adoption and Waiver of Rights 
UT Notice of Adoption and Notice of Rights for Publication 
UT Notice of Petition to Adopt and Notice of Rights 
UT Notice of Petition to Adopt and Notice of Rights 
UT Petition to Adopt a Minor Stepchild 
UT Proof of Alternative Service 
UT Proof of Completed Service 
UT Request to Submit for Decision 
UT Utah Distric

UT Three day notice to vacate for assigning or subletting contrary to rental contract 
UT Three day notice to vacate for committing criminal act on the premises 
UT Three day notice to vacate for committing waste on premises 
UT Three day notice to vacate for criminal nuisance 
UT Three day notice to vacate for engaging in unlawful business on or in the premises 
UT Three day notice to vacate for lease violation which cannot be brought into compliance 
UT Three day notice to vacate for non-criminal nuisance 
UT Acceptance of Service – Expungement 
UT Consent and Waiver of Hearing – Expungement 
UT Notice of Hearing – Expungement 
UT Order on Petition to Expunge Records 
UT Order on Petition to Expunge Records 
UT Order on Petition to Expunge Records 
UT Order on Petition to Expunge Records 
UT Order on Petition to Expunge Records 
UT Order on Petition to Expunge Records 
UT Petition to Expunge Records 
UT Petition to Expunge Records 
UT Petition to Expunge Records 
UT Petition to Expun

UT Motion to appear remotely - district and justice 
UT Motion to appear remotely - juvenile 
UT Order on motion to appear remotely - district and justice 
UT Motion to change venue 
UT Order on motion to change venue 
UT Motion to continue hearing or trial 
UT Order on motion to continue hearing or trial 
UT Motion to Correct Clerical Mistake 
UT Order on motion to correct clerical mistake 
UT Motion to delay 
UT Order on motion to delay enforcement of judgment 
UT Ex Parte Verified Motion to Enforce Domestic Order and for Sanctions 
UT Ex Parte Verified Motion to Enforce Domestic Order and for Sanctions - Commissioner 
UT Ex Parte Verified Motion to Enforce Order and for Sanctions 
UT Order on Motion to Enforce Domestic Order 
UT Order on Motion to Enforce Order 
UT Order to Attend Hearing 
UT Request for Contempt Hearing 
UT Motion to excuse mandatory divorce mediation 
UT Motion to excuse mediation 
UT Order on motion to excuse mandatory divorce mediation 
UT Order on motion to exc

UT Order Denying Temporary Dating Violence Protective Order 
UT Order Denying Temporary Protective Order 
UT Order Denying Temporary Sexual Violence Protective Order 
UT Order Dismissing Request for Protective Order 
UT Order Dismissing Request for Sexual Violence Protective Order 
UT Order Extending Sexual Violence Protective Order 
UT Order Extending Temporary Protective Order 
UT Order on Request to Dismiss or Vacate Protective Order 
UT Order to Wireless Service Provider to Transfer Phone Number
UT Petitioner's Request to Dismiss Protective Order  
UT Protective Order 
UT Request for Dating Violence Protective Order 
UT Request for Hearing on Protective Order 
UT Request for Protective Order 
UT Request for Sexual Violence Protective Order 
UT Request to Dismiss Sexual Violence Protective Order 
UT Request to Extend Sexual Violence Protective Order 
UT Request to Modify Protective Order 
UT Request to Modify Sexual Violence Protective Order 
UT Respondent’s Request to Dismiss Prote

MI Answer, Nonpayment of Rent, Landlord-Tenant
MI Answer, Nonpayment of Rent, Landlord-Tenant
MI Answer, Termination of Tenancy, Mobile Home Park - Mobile Home Owner (Just-Cause Termination)
MI Answer, Termination of Tenancy, Mobile Home Park - Mobile Home Owner (Just-Cause Termination)
MI Application and Order of Eviction, Landlord-Tenant/Land Contract
MI Complaint for Possession after Land Contract Forfeiture
MI Complaint to Recover Possession of Property
MI Complaint, Damage/Health Hazard to Property, Landlord-Tenant
MI Complaint, Nonpayment of Rent, Landlord-Tenant
MI Complaint, Termination of Tenancy, Mobile Home Park - Mobile Home Owner (Just-Cause Termination)
MI Consent Order for Conditional Dismissal, Landlord-Tenant
MI Demand for Possession, Damage/Health Hazard to Property, Landlord-Tenant
MI Demand for Possession, Damage/Health Hazard to Property, Landlord-Tenant (two-page form)
MI Demand for Possession, Nonpayment of Rent, Landlord-Tenant
MI Demand for Possession, Nonpayme

MI Claim of Appeal
MI Claim of Appeal and Order Appointing Appellate Counsel
MI Claim of Appeal and Order Appointing Counsel
MI Claim of Appeal on Application for Concealed Pistol License
MI Docketing Statement (Court of Appeals)
MI Jurisdictional Checklist for Court of Appeals
MI MAACS Statement of Service and Order for Payment of Court Appointed Counsel
MI Motion to Change Time for Filing Transcript on Appeal
MI Notice of Filing of Transcript and Affidavit of Mailing
MI Notice of Intent to Dismiss Appeal (formerly MC 59)
MI Notice of Record Return from Circuit Court/Court of Appeals
MI Notice of Right to Appellate Review and Request for Appointment of Attorney
MI Notice to Prisoner on Application for Leave to Appeal Decision of Parole Board
MI Order Denying Appointment of Appellate Counsel
MI Order Dismissing Appeal (formerly MC 61)
MI Order Following Appeal on Application for Concealed Pistol License
MI Order on Application for Leave to Appeal
MI Order Regarding Appointment of Appel

MI Proof of Service
MI Publication of Notice
MI Publication of Notice of Hearing
MI Receipt of Distributive Share, Personal Property
MI Receipt of Property from Conservator
MI Register’s Statement
MI Registration of Trust
MI Renunciation of Right to Appointment, Nomination of Personal Representative and Waiver of Notice
MI Safe-Deposit Box Certificate and Receipt
MI Schedule of Distributions and Payment of Claims
MI Selection of Homestead Allowance and Exempt Property
MI Statement and Proof of Claim
MI Supplemental Testimony to Identify Nonheir Devisees, Testate Estate
MI Sworn Closing Statement, Summary Proceeding,  Small Estates
MI Sworn Statement to Close Unsupervised Administration
MI Testimony to Identify Heirs
MI Waiver/Consent
MI Clerk's Certificate of Sale of Real Estate Pursuant to Judgment
MI County Clerk's Report of Sale
MI Notice of Foreclosure Sale
MI Assignment to Youthful Trainee Status
MI Commitment Order, Not Guilty by Reason of Insanity
MI Judgment of Sentence, Commit

MI Motion for Alternate Service
MI Motion Re Transfer/Hearing/Placement and Authorization/Denial/Notice of Hearing
MI Motion to Waive Jurisdiction and Notice
MI Notice of Diversion Conference
MI Notice of Hearing
MI Notice of Hearing on Probation Violation
MI Notice of Intent to Intercept State Income Tax
MI Notice of Proceedings Concerning An Indian Child
MI Notice to Putative Father
MI Order After Dispositional Review/Permanency Planning Hearing (Child Protective Proceedings)
MI Order After First-Phase Hearing to Waive Jurisdiction (Delinquency Proceedings)
MI Order After Hearing on Competency (Delinquency Proceedings)
MI Order After Posttermination Review/Permanency Planning Hearing (Child Protective Proceedings)
MI Order after Preliminary Hearing (Child Protective Proceedings)
MI Order After Preliminary Hearing/Inquiry (Delinquency/Personal Protection)
MI Order After Pretrial Hearing (Child Protective Proceedings)
MI Order After Removal Hearing (Child Protective Proceedings)
MI Ord

MI Order Suspending Immediate Income Withholding
MI Order Suspending License (two-part form)
MI Order to Release Lien
MI Order to Remit Prisoner Funds for Child Support - FOC 112
MI Order Vacating/Confirming Registration of Out-of-State Support Order
MI Petition (Collaborative Law Process)
MI Petition (Consent Judgment)
MI Prisoner Information Addendum
MI Publication and Notice of Friend of the Court Annual Statutory Review
MI Referee Findings and Recommendation for Order after Hearing on Modification of Support
MI Referee Findings and Recommendation for Order after Hearing on Bench Warrant/Show Cause (Support)
MI Referee Findings and Recommendation for Order after Hearing on Alleged Custody/Parenting Time Violation
MI Referral Order
MI Request for Health-Care Expense Payment
MI Request to Access Friend of the Court Records and Decision
MI Request to Reopen Friend of the Court Case
MI Response to Motion Regarding Change of Domicile/Legal Residence (with instructions)
MI Response to Mot

MI Order Following Hearing on Objection By Minor to Hospitalization/Administrative Admission
MI Order Following Hearing on Petition for Continued Hospitalization of Minor
MI Order Following Hearing on Petition for Discharge from Judicial Admission
MI Order Following Hearing on Petition for Judicial Admission
MI Order for Examination/Transport
MI Order for Report after Notification and Report
MI Order for Report on Petition for Judicial Admission
MI Order Regarding Transport of Minor
MI Order to Modify Order for Assisted Outpatient Treatment or Combined Hospitalization and Assisted Outpatient Treatment
MI Petition for Continued Hospitalization of Minor
MI Petition for Continuing Mental Health Treatment Order
MI Petition for Discharge from Continuing Mental Health Treatment
MI Petition for Discharge from Judicial Admission
MI Petition for Judicial Admission
MI Petition for Mental Health Treatment
MI Petition for Second Mental Health Treatment Order
MI Petition Regarding Transport of Mino